`Дисциплина: Методы и технологии машинного обучения`   
`Уровень подготовки: бакалавриат`   
`Направление подготовки: 01.03.02 Прикладная математика и информатика`   
`Семестр: осень 2021/2022`   

In [ ]:
# настройка ширины страницы блокнота .......................................
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# расширение watermark для вывода информации о версиях пакетов
#  https://github.com/rasbt/watermark
%load_ext watermark

# Лабораторная работа №5: Методы, основанные на деревьях решений. Регрессионные деревья. Деревья классификации. Случайный лес. Бустинг.  

В практических примерах ниже показано:   

* как делать перекодировку признаков в номинальной и порядковой шкалах
* как вырастить дерево и сделать обрезку его ветвей   
* как настроить модель бэггинга   
* как вырастить случайный лес  
* как настроить модель бустинга на деревьях решений  
* как подбирать настроечные параметры моделей методом сеточного поиска  

Точность всех моделей оценивается методом перекрёстной проверки по 5 блокам.  

*Модели*: дерево классификации, бэггинг, случайный лес, бустинг, дерево регрессии  
*Данные*: `in-vehicle-coupon-recommendation.csv`. Источник: [сайт Калифорнийского университета в Ирвине](https://archive.ics.uci.edu/ml/datasets/in-vehicle+coupon+recommendation)

In [ ]:
# выводим информацию о версиях python и пакетов
%watermark -a "aksyuk@github.com" -d -v -p numpy,pandas,matplotlib,sklearn 

# Указания к выполнению


## Загружаем пакеты

In [ ]:
# загрузка пакетов: инструменты --------------------------------------------
#  работа с массивами
import numpy as np
#  фреймы данных
import pandas as pd
#  графики
import matplotlib as mpl
#  стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# загрузка файлов по URL
import urllib
# проверка существования файла на диске
from pathlib import Path
# для форматирования результатов с помощью Markdown
from IPython.display import Markdown, display
# перекодировка категориальных переменных
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# хи-квадрат тест на независимость по таблице сопряжённости
from scipy.stats import chi2_contingency
#  для таймера
import time

# загрузка пакетов: данные -------------------------------------------------
from sklearn import datasets

# загрузка пакетов: модели -------------------------------------------------
#  дерево классификации
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# бэггинг
from sklearn.ensemble import BaggingClassifier
# случайный лес
from sklearn.ensemble import RandomForestClassifier
# бустинг
from sklearn.ensemble import GradientBoostingClassifier
#  сводка по точности классификации
from sklearn.metrics import classification_report

In [ ]:
# константы
#  ядро для генератора случайных чисел
my_seed = 9212
#  создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
#  примеры стилей и шаблонов графиков: 
#  http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

In [ ]:
# функция форматирования результатов с использованием Markdown
def printmd(string):
    display(Markdown(string))
    
# функции для попарной конкатенации элементов двух списков
concat_func_md = lambda x, y: '`' + str(x) + "`:&ensp;&ensp;&ensp;&ensp;" + str(y)
concat_func = lambda x, y: str(x) + ' ' * 4 + str(y)


# функция, которая строит график важности признаков в модели случайного леса
#  источник: https://www.analyseup.com/learn-python-for-data-science/python-random-forest-feature-importance-plot.html
def plot_feature_importance(importance, names, model_type) :
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,
                      inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title('Важность признаков в модели: ' + model_type)
    plt.xlabel('Важность признака')
    plt.ylabel('')

## Загружаем данные

Набор данных можно загрузить напрямую по ссылке: <https://raw.githubusercontent.com/aksyuk/MTML/main/Labs/data/in-vehicle-coupon-recommendation.csv>. Справочник к данным доступен по адресу: <https://github.com/aksyuk/MTML/blob/main/Labs/data/CodeBook_in-vehicle-coupon-recommendation.md>.    

Загружаем данные во фрейм и выясняем их размерность. В таблице много строк, поэтому для экономии времени загрузку сделаем в два шага: сначала скачаем таблицу и сохраним в папку `'./data'`, затем прочитаем её во фрейм. Перед скачиванием проверим, нет ли уже такого файла в папке с данными.  

In [ ]:
# путь к локальному файлу для сохранения
localFilePath = './data/in-vehicle-coupon-recommendation.csv'

# проверяем, нет ли уже такого файла на диске

    # загружаем таблицу и превращаем её во фрейм
    fileURL = 'https://raw.githubusercontent.com/aksyuk/MTML/main/Labs/data/in-vehicle-coupon-recommendation.csv'
    # скачиваем
    
    print('Файл', localFilePath,'успешно загружен с адреса ', fileURL, '\n')
else:
    print('Файл', localFilePath,'уже есть на диске\n')

# читаем
DF_raw = 

# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

In [ ]:
# типы столбцов
DF_raw.dtypes

Проблема в том, что, судя по справочнику к данным, все столбцы таблицы являются категориальными. Однако некоторые (бинарные) воспринимаются как `int`, а остальные как `object`. Посмотрим на столбцы типа `int`.   

In [ ]:
# первые 7 строк столбцов типа int64


Функция построения дерева классификации `DecisionTreeClassifier()` требует числовых порядковых значений переменных. Видно, что столбцы типа `int64` либо порядковые (`temperature`), либо бинарные (все остальные), их преобразовывать нет необходимости. А вот столбцы типа `object` придётся кодировать вручную.  
При этом на этапе предварительного анализа данных нам удобнее будет работать с исходными категориальными столбцами. Поэтому сейчас просто изменим тип столбцов `object` на `category`.  

In [ ]:
# меняем тип столбцов на категориальные



Отложим 30% наблюдений для прогноза.  

In [ ]:
# наблюдения для моделирования
DF = 
# отложенные наблюдения
DF_predict = 

# Предварительный анализ данных  

## Описательные статистики  

Стандартный подсчёт статистик с помощью фунции `describe()` бесполезен для категориальных столбцов, поэтому рассчитаем частоты категорий по каждому столбцу. Для вывода отчёта воспользуемся форматированием на Markdown.   

In [ ]:
# считаем частоты по столбцам, учитывая пропуски
for col in DF.columns:
    freq_col = 
    str_freqs = 
    str_names = 
    # для вывода в html
    printmd('**' + col + '**</br>' + 
            '</br>'.join(list(map(concat_func_md, str_names, str_freqs))))
    # для сохранения в pdf
    # print('\n', col, '\n', 
    #       '\n'.join(list(map(concat_func, str_names, str_freqs))))

Обратим внимание на столбцы `car` и `toCoupon_GEQ5min`, которые есть в таблице, но отсутствовали в справочнике к данным. В первом (тип автомобиля) пропущено 99,1% наблюдений, во втором (до ресторана/кофейни, в которую выдан купон, более 5 минут езды) значения во всех наблюдениях одинаковы. Уберём эти столбцы из обучающих и отложенных данных.   

In [ ]:
# выбрасываем стобцы с большинством пропусков или с нулевой дисперсией
#  из обучающей выборки
DF = 
#  и из отложенных наблюдений
DF_predict = 

Ещё раз оценим количество пропусков.  

In [ ]:
# считаем пропуски в столбцах, выводим ненулевые значения
nas = DF.isna().sum()
nas


Подсчитаем, сколько наблюдений мы потеряем, если выбросим все строки хотя бы с одним пропуском.  

In [ ]:
na_rows = sum([True for idx, row in DF.iterrows() if any(row.isnull())])
print('Из-за пропусков пропадает ', na_rows, ' строк (',
      np.around(na_rows / DF.shape[0] * 100, 1), '%)', sep='')

Выводы по описательным статистикам: доли классов (`Y`) сопоставимы, наибольшее количество категорий у объясняющей переменной `occupation`. Строки с пропусками составляют не более 5%, поэтому мы уберём их из обучающей выборки.  

In [ ]:
# выкидываем пропуски из обучащей
DF = 
DF.shape

In [ ]:
# выкидываем пропуски из отложенных наблюдений
DF_predict = 
DF_predict.shape

## Распределение предикторов внутри классов  по зависимой переменной

Все объясняющие переменные являются категориальными, поэтому оценивать их связь с зависимой переменной с помощью корреляционной матрицы некорректно. Вместо этого можно воспользоваться [критерием согласия Хи-квадрат](https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%B8%D1%82%D0%B5%D1%80%D0%B8%D0%B9_%D1%81%D0%BE%D0%B3%D0%BB%D0%B0%D1%81%D0%B8%D1%8F_%D0%9F%D0%B8%D1%80%D1%81%D0%BE%D0%BD%D0%B0), который рассчитывается по таблице сопряжённости. Нулевая гипотеза теста: распределение долей в таблице сопряжённости случайно, т.е. два показателя независимы друг от друга.     
Проведём тест для всех пар "объясняющая переменная" – "зависимая переменная" и выведем те пары, для которых соответствующее критерию p-значение больше 0.05 (т.е. нулевая гипотеза принимается, переменные независимы).  

In [ ]:
for col in DF.columns[:24] :
    con_tab = 
    c, p, dof, expected = 
    if p > 0.05 :
        print(col, 'и Y',
              '\nH_0: переменные распределены независимо друг от друга', 
              '\nP-значение:', np.around(p, 4))

Интересный результат: полное совпадение p-значений – объясняется тем, что на самом деле `direction_same` и `direction_opp` противоположны друг другу. Связь между ними функциональная: если направление на ресторан/кофейню, в который предлагается купон, не совпадает с направлением на исходное место назначения (`direction_same == 0`), то оно противоположно (`direction_opp == 1`), и наоборот. Поэтому в модель имеет смысл включать только одну из этих переменных.   

In [ ]:
# исключаем direction_opp 
#  из обучающей выборки
DF = DF.drop(['direction_opp'], axis=1)
#  и из отложенных наблюдений
DF_predict = DF_predict.drop(['direction_opp'], axis=1)

## Перекодировка номинальной и порядковой шкалы   

Теперь перекодируем признаки так, чтобы воспользоваться функцией классификации на дереве решений. Начнём с тех, которые содержат признаки в номинальной шкале (между позициями нет отношения порядка). Перекодируем их в фиктивные с помощью функции `OneHotEncoder()`.   

In [ ]:
# имена столбцов с номинальными показателями
nom_col_names = ['destination', 'passanger', 'weather', 'coupon', 'gender', 
                 'maritalStatus', 'occupation']

# создаём объект кодировщика
one_hot = OneHotEncoder()

# кодируем, результат – массив
recoded = 

# создаём из результата новый фрейм с фиктивными переменными
clmns = 
df_dummy_nom = 

# выводим размерность итога
print(df_dummy_nom.shape)

# смотрим результат
df_dummy_nom.head()

In [ ]:
# исходник для сравнения
print(DF[nom_col_names].shape)
DF[nom_col_names].head()

Теперь разбираемся с показателями в порядковой шкале. Для этого воспользуемся `OrdinalEncoder()`. Для начала убедимся, что на этапе исключения пропущенных всё прошло штатно, и значений `'nan'`, которые `OrdinalEncoder()` не умеет обрабатывать, не осталось.  

In [ ]:
# имена столбцов с порядковыми показателями
ord_col_names = ['time', 'expiration', 'age', 'education', 'income', 
                 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20',
                 'Restaurant20To50']

# считаем пропуски в столбцах
for col in ord_col_names :
    print('Пропусков в столбце', col, ':',
          sum(DF[col].isnull().astype(int)))

Всё отлично, пропусков нет, поэтому можно перекодировать все порядковые столбцы в одно действие.   

In [ ]:
# создаём списки с порядком кодировки для каждого столбца
enc_time = ['7AM', '10AM', '2PM', '6PM', '10PM']
enc_expiration = ['2h', '1d']
enc_age = ['below21', '21', '26', '31', '36', '41', '46', '50plus']
enc_education = ['Some High School', 'High School Graduate', 
                 'Some college - no degree', 'Associates degree',
                 'Bachelors degree', 
                 'Graduate degree (Masters or Doctorate)']
enc_income = ['Less than $12500', '$12500 - $24999', '$25000 - $37499',
              '$37500 - $49999', '$50000 - $62499', '$62500 - $74999',
             '$75000 - $87499', '$87500 - $99999', '$100000 or More']
enc_how_often = ['never', 'less1', '1~3', '4~8', 'gt8']

# перекодировщик
ordinal = 

# кодируем
df_ord = 

# выводим размерность итога
print(df_ord.shape)

# результат
df_ord.head()

In [ ]:
# исходник для сравнения
print(DF[ord_col_names].shape)
DF[ord_col_names].head()

Объединим результаты: исходно числовые столбцы, дамми для признаков в номинальной шкале и перекодированные признаки в порядковой шкале – во фрейм под названием `DF_num`.  

In [ ]:
# объединяем результаты перекодировки в один фрейм
DF_num = 


print('Размерность обучающего фрейма после исключения NaN',
      '\nи перекодировки: ', DF_num.shape)

# результат
DF_num.head()

Повторяем перекодировку для фрейма с отложенными наблюдениями `DF_predict`.   

In [ ]:
# перекодировка отложенных наблюдений
#  номинальная шкала -------------------------------------------------------
#   кодируем, результат – массив
recoded = one_hot.fit_transform(DF_predict[nom_col_names]).toarray()

# создаём из результата новый фрейм с фиктивными переменными
clmns = one_hot.get_feature_names(nom_col_names)
df_dummy_nom = pd.DataFrame(recoded, columns=clmns)

#  порядковая шкала --------------------------------------------------------
#   кодируем
df_ord = pd.DataFrame(ordinal.fit_transform(DF_predict[ord_col_names]), 
                        columns = ord_col_names)

#   объединяем результаты
DF_predict_num = pd.concat([DF_predict.loc[:, 
    DF_predict.dtypes == 'int64'].reset_index(), 
                            df_dummy_nom, df_ord], axis=1)

print('Размерность фрейма с отложенными наблюдениями после исключения NaN',
      '\nи перекодировки: ', DF_predict_num.shape)

# результат
DF_predict_num.head()

# Модель дерева  

В этом разделе построим:  

* дерево классификации  
* дерево классификации с обрезкой ветвей  


## Дерево на всех признаках    

Построим модель и выведем изображение дерева в виде текста.  

In [ ]:
# выращиваем дерево на всех объясняющих
X = 
y = 

# классификатор
cls_one_tree = 

tree_full = 

# выводим количество листьев (количество узлов)


In [ ]:
# глубина дерева: количество узлов от корня до листа
#  в самой длинной ветви


Очевидно, дерево получилось слишком большое для отображения в текстовом формате. Графическая визуализация тоже не поможет в данном случае. Посчитаем показатели точности с перекрёстной проверкой.   

In [ ]:
# будем сохранять точность моделей в один массив:
score = list()
score_models = list()

# считаем точность с перекрёстной проверкой, показатель Acc
cv = 


# записываем точность
score.append(np.around(np.mean(cv), 3))
score_models.append('one_tree')

print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[0], ':', score[0])

## Дерево с обрезкой ветвей   

Подберём оптимальное количество ветвей, которое максимизирует $Acc$, для экономии времени рассчитанный методом проверочной выборки.  

In [ ]:
# рассчитываем параметры alpha для эффективных вариантов обрезки ветвей
path = 
ccp_alphas, impurities = path.ccp_alphas, path.impurities
print('Всего значений alpha:', len(ccp_alphas))
print('Энтропия листьев для первых 5 значений alpha:', impurities[:5])

In [ ]:
# изображаем на графике
plt.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
plt.xlabel("значение гиперпараметра alpha")
plt.ylabel("общая энтропия листьев дерева")
plt.title("Изменение показателя нечистоты узлов с ростом alpha")
plt.show()

In [ ]:
# обучающая и тестовая выборки, чтобы сэкономить время
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    random_state=my_seed)

# модели
clfs = list()

# таймер
tic = time.perf_counter()
# цикл по значениям alpha
for ccp_alpha in ccp_alphas:
    clf = 
    
    

# таймер
toc = time.perf_counter()
print(f"Расчёты по обрезке дерева заняли {toc - tic:0.2f} секунд")

In [ ]:
# извлекаем характеристики глубины и точности
#  таймер
tic = time.perf_counter()
node_counts = 
train_scores = 
test_scores = 
#  таймер
toc = time.perf_counter()
print(f"Расчёты показателей точности заняли {toc - tic:0.2f} секунд")

In [ ]:
# изображаем на графике
fig, ax = plt.subplots(1, 2)

# график глубины дерева
ax[0].plot(ccp_alphas, node_counts, marker=',', drawstyle="steps-post")
ax[0].set_xlabel("значение гиперпараметра alpha")
ax[0].set_ylabel("количество узлов")
ax[0].set_title("Сложность модели vs alpha")

# график точности
ax[1].plot(ccp_alphas, train_scores, marker=',', label='train',
           drawstyle="steps-post")
ax[1].plot(ccp_alphas, test_scores, marker=',', label='test',
           drawstyle="steps-post")
ax[1].set_xlabel("значение гиперпараметра alpha")
ax[1].set_ylabel("Acc")
ax[1].set_title("Точность модели  vs alpha")
fig.tight_layout()

Находим оптимальный размер дерева по максимуму $Acc$ на тестовой выборке.  

In [ ]:
# оптимальное количество узлов
opt_nodes_num = 

# считаем точность с перекрёстной проверкой, показатель Acc
cv = 

# записываем точность
score.append(np.around(np.mean(cv), 3))
score_models.append('pruned_tree')

print('Оптимальное количество узлов:', opt_nodes_num,
      '\nсоответствующая Acc на тествоой:', np.around(max(test_scores), 3),
      '\n\nAcc с перекрёстной проверкой',
      '\nдля модели', score_models[1], ':', score[1])

Посмотрим на характеристики глубины и сложности построенного дерева с обрезкой ветвей.

In [ ]:
# выводим количество листьев (количество узлов)
clfs[opt_nodes_num].get_n_leaves()

In [ ]:
# глубина дерева: количество узлов от корня до листа
#  в самой длинной ветви
clfs[opt_nodes_num].get_depth()

---

📚 **Пример визуализации небольшого дерева**

Лучшее дерево с обрезкой по-прежнему слишком велико для визуализации. Для примера нарисуем одно из небольших деревьев с обрезкой и выведем его же в виде текста.  

In [ ]:
# находим деревья с количеством листьев меньше 20


In [ ]:
# визуализация на схеме НА ПРИМЕРЕ МАЛЕНЬКОГО ДЕРЕВА
nodes_num = 
print('Количество узлов:', nodes_num,
      '\nТочность дерева на тестовой:', 
      np.around(test_scores[node_counts.index(nodes_num)], 3))

fig = plt.figure(figsize=(25,20))
_ = plot_tree(, 
              filled=True)

In [ ]:
# визуализируем дерево в виде текстовой схемы
viz = export_text(, 
                  feature_names=list(X.columns))
print(viz)

---

# Бэггинг  

Модель бэггинга использует бутстреп, чтобы вырастить $B$ деревьев на выборках с повторами из обучающих данных. Построим модель для $B=50$ деревьев.  

In [ ]:
# параметр B: количество деревьев
num_trees = 50

# разбиения для перекрёстной проверки
kfold = 

# таймер
tic = time.perf_counter()
# модель с бэггингом
tree_bag = 

cv = 

# таймер
toc = time.perf_counter()
print(f"Обучение модели с бэггингом на {num_trees:0.0f} деревьях", 
      " и перекрёстной проверкой ", 
      f"заняло {toc - tic:0.2f} секунд", sep='')

In [ ]:
# точность
np.around(np.mean(cv), 3)

Итак, мы построили модель, выбрав параметр $B$ случайным образом. Воспользуемся функцией `GridSearchCV()`, чтобы перебрать 5 вариантов значений для параметра $B$.  

In [ ]:
# настроим параметры бэггинга с помощью сеточного поиска
param_grid = 

# таймер
tic = time.perf_counter()
clf = 

tree_bag = 
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

In [ ]:
# точность лучшей модели
np.around(, 3)

In [ ]:
# количество деревьев у лучшей модели


Таким образом, перебрав несколько вариантов для $B$, мы немного улучшили первоначальную точность модели бэггинга.  

In [ ]:
# записываем точность
score.append(np.around(tree_bag.best_score_, 3))
score_models.append('bagging_GS')

print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[2], ':', score[2])

# Случайный лес  



У модели случайного леса два настроечных параметра: количество деревьев $B$ и количество признаков для построения отдельного дерева $m$. Настроим сеточный поиск для их подбора.  

In [ ]:
# сколько столбцов в обучающих данных (p)
X_m = X.shape[1]
# возьмём значения для m: p, p/2, sqrt(p) и log2(p)
ms = np.around([X_m, X_m / 2, np.sqrt(X_m), np.log2(X_m)]).astype(int)
ms

In [ ]:
# настроим параметры случайного леса с помощью сеточного поиска
param_grid = {'n_estimators' : [10, 20, 30, 40, 50],
              'max_features' : ms}

# таймер
tic = time.perf_counter()
clf = GridSearchCV(RandomForestClassifier(DecisionTreeClassifier()),
                   param_grid, scoring='accuracy', cv=kfold)
random_forest = clf.fit(X, y)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

In [ ]:
# точность лучшей модели
np.around(random_forest.best_score_, 3)

In [ ]:
# количество деревьев у лучшей модели
random_forest.best_estimator_.get_params()['n_estimators']

In [ ]:
# количество объясняющих у лучшей модели
random_forest.best_estimator_.get_params()['max_features']

In [ ]:
# рисуем график относительной важности каждого признака
plot_feature_importance(random_forest.best_estimator_.feature_importances_,
                        X.columns, 'Случайный лес')

In [ ]:
# записываем точность
score.append(np.around(random_forest.best_score_, 3))
score_models.append('random_forest_GS')

print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[3], ':', score[3])

# Бустинг 



Подберём сеточным поиском настроечные параметры модели:  
* $B$ число деревьев, 
* $\lambda$ – скорость обучения,
* $d$ – глубина взаимодействия предикторов.

In [ ]:
# обучаем модель с параметрами по умолчанию
clf_tst = 
cv = cross_val_score(clf_tst, X, y, cv=kfold, scoring='accuracy')
np.around(np.mean(cv), 3)

In [ ]:
# настроим параметры бустинга с помощью сеточного поиска
param_grid = {'n_estimators' : [10, 20, 30, 40, 50],
              'learning_rate' : np.linspace(start=0.01, stop=0.25, num=15),
              'max_depth' : [1, 2]}

# таймер
tic = time.perf_counter()
clf = GridSearchCV(GradientBoostingClassifier(),
                   param_grid, scoring='accuracy', cv=kfold)
boost_tree = clf.fit(X, y)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

In [ ]:
# точность лучшей модели
np.around(boost_tree.best_score_, 3)

In [ ]:
# параметры лучшей модели
print('n_estimators:', 
      boost_tree.best_estimator_.get_params()['n_estimators'],
      '\nlearning_rate:',
      boost_tree.best_estimator_.get_params()['learning_rate'],
      '\nmax_depth:',
      boost_tree.best_estimator_.get_params()['max_depth'])

In [ ]:
# записываем точность
score.append(np.around(boost_tree.best_score_, 3))
score_models.append('boost_tree_GS')

print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[4], ':', score[4])


# Прогноз на отложенные наблюдения по лучшей модели

Ещё раз посмотрим на точность построенных моделей.  

In [ ]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

Все модели показывают среднюю точность по показателю $Acc$, при этом самой точной оказывается модель случайного леса. Сделаем прогноз на отложенные наблюдения.   

In [ ]:
# данные для прогноза
X_pred = 
# строим прогноз
y_hat = 
# характеристики точности


# Источники 

1. Джеймс Г., Уиттон Д., Хасти Т., Тибширани Р. Введение в статистическое обучение с примерами на языке R. Пер. с англ. С.Э. Мастицкого – М.: ДМК Пресс, 2016 – 450 с.  
1. *Рашка С.* Python и машинное обучение: крайне необходимое пособие по новейшей предсказательной аналитике, обязательное для более глубокого понимания методологии машинного обучения / пер. с англ. А.В. Логунова. – М.: ДМК Пресс, 2017. – 418 с.: ил.  
1. *Tong Wang*, *Cynthia Rudin*, *Finale Doshi-Velez*, *Yimin Liu*, *Erica Klampfl*, *Perry MacNeille* A Bayesian Framework for Learning Rule Sets for Interpretable Classification / Journal of Machine Learning Research 18 (2017) 1-37. URL: <https://jmlr.org/papers/volume18/16-003/16-003.pdf>  
1. *George Pipis* How to Run the Chi-Square Test in Python / medium.com. URL: <https://medium.com/swlh/how-to-run-chi-square-test-in-python-4e9f5d10249d>   
1. *Bernd Klein* What are Decision Trees? / python-course.eu. URL: <https://www.python-course.eu/Decision_Trees.php>  
1. Pruning decision trees - tutorial / kaggle.com. URL: <https://www.kaggle.com/arunmohan003/pruning-decision-trees-tutorial>  
1. Post pruning decision trees with cost complexity pruning / scikit-learn.org. URL: <https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html>  
1. *Piotr Płoński* Visualize a Decision Tree in 4 Ways with Scikit-Learn and Python / mljar.com. URL: <https://mljar.com/blog/visualize-decision-tree/>  
1. Random Forest Feature Importance Plot / www.analyseup.com. URL: <https://www.analyseup.com/learn-python-for-data-science/python-random-forest-feature-importance-plot.html>  